# Evaluating RAG Pipelines with RAGAS

Welcome to this tutorial on evaluating Retrieval-Augmented Generation (RAG) pipelines using the RAGAS framework!  

**RAG** is a powerful technique that combines the strengths of large language models (LLMs) with external knowledge bases. This allows LLMs to provide answers that are not only fluent and coherent but also grounded in factual, up-to-date information.

However, building a robust RAG system can be challenging. How do you know if your retriever is fetching the right context? Is the generator producing answers that are faithful to the context? This is where **RAGAS** comes in.

**RAGAS** is an open-source framework that helps you evaluate your RAG pipelines. It provides a set of metrics to assess the performance of both the retrieval and generation components.

## Setup

In [2]:
#%pip install -qU ragas langchain-openai langchain datasets python-dotenv

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set your API keys
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Key RAGAS Metrics

RAGAS provides several key metrics to evaluate different aspects of your RAG pipeline. Here are some of the most important ones:

### Retrieval Metrics

- **Context Precision:** Measures the signal-to-noise ratio of the retrieved context. A high score means the retrieved context is relevant to the question.
- **Context Recall:** Measures the ability of the retriever to fetch all the necessary information to answer the question.

### Generation Metrics

- **Faithfulness:** Measures the factual consistency of the generated answer with the retrieved context. A low score indicates the answer is hallucinated.
- **Answer Relevancy:** Measures how relevant the generated answer is to the question.

## Building a Simple RAG Pipeline

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# For this example, we'll use a simple, hardcoded context.
# In a real-world scenario, this would be the output of your retriever.
context = ["The first Olympic Games were held in ancient Greece in 776 BC."]

llm = ChatOpenAI(model="gpt-3.5-turbo")

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Use the following context to answer the question: {context}"),
        ("human", "{question}"),
    ]
)

rag_chain = prompt_template | llm | StrOutputParser()

## Preparing the Evaluation Dataset

RAGAS needs a dataset to evaluate your pipeline. This dataset should contain questions, the generated answers, the retrieved contexts, and the ground truth answers.

For this tutorial, we will create a small, synthetic dataset.

In [5]:
from datasets import Dataset

questions = [
    "When were the first Olympic Games held?",
    "Where were the first Olympic Games held?"
]
ground_truths = [
    "The first Olympic Games were held in 776 BC.",
    "The first Olympic Games were held in ancient Greece."
]

answers = []
contexts = []

for question in questions:
    answers.append(rag_chain.invoke({"context": context, "question": question}))
    contexts.append(context)

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths,
}

dataset = Dataset.from_dict(data)

## Running the Evaluation

In [6]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
)

df = result.to_pandas()
df

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

user_input  \
0   When were the first Olympic Games held?   
1  Where were the first Olympic Games held?   

                                  retrieved_contexts  \
0  [The first Olympic Games were held in ancient ...   
1  [The first Olympic Games were held in ancient ...   

                                            response  \
0  The first Olympic Games were held in ancient G...   
1  The first Olympic Games were held in ancient G...   

                                           reference  faithfulness  \
0       The first Olympic Games were held in 776 BC.           1.0   
1  The first Olympic Games were held in ancient G...           1.0   

   answer_relevancy  context_recall  context_precision  
0          1.000000             1.0                1.0  
1          0.999999             1.0                1.0

## Interpreting the Results

The output dataframe will show the scores for each metric for every question in your dataset. A score of 1.0 is the best possible score, while a score of 0.0 is the worst.

By analyzing these scores, you can identify the strengths and weaknesses of your RAG pipeline. For example:

- Low **faithfulness** scores might indicate that your generator is hallucinating.
- Low **context_recall** could mean your retriever is not finding all the relevant information.
- Low **context_precision** might suggest that your retriever is fetching irrelevant context, which could confuse the generator.
- Low **answer_relevancy** may point to issues with how the generator is using the context to form an answer.

## Conclusion

RAGAS is an invaluable tool for anyone building RAG pipelines. By providing a quantitative way to measure performance, it allows you to systematically improve your system. This tutorial has covered the basics of setting up RAGAS, understanding its core metrics, and running an evaluation. From here, you can explore more advanced features like synthetic test data generation and custom metrics.